In [3]:
import pandas as pd

# Step 1: Load patient data
patient_data = pd.read_csv('patient_data.csv', encoding='latin1')

# Step 2: Clean patient data (keep needed columns and make a full copy)
patient_data_cleaned = patient_data[['pid', 'fname', 'lname', 'DOB', 'sex', 'city', 'state']].copy()

# Step 3: Create location data (no dropping duplicates)
location_data = patient_data_cleaned[['city', 'state']].copy()

# Step 4: Add LocationID (one for each patient)
location_data['LocationID'] = location_data.index + 1

# Step 5: Reorder location columns
location_data = location_data[['LocationID', 'city', 'state']]

# Step 6: Add LocationID to patient data
patient_data_cleaned['LocationID'] = location_data['LocationID']

# Step 7: Finalize patient table (only needed columns)
patient_final = patient_data_cleaned[['pid', 'fname', 'lname', 'DOB', 'sex', 'LocationID']]

# Step 8: Save the results
patient_final.to_csv('cleaned_patient_data.csv', index=False)  # Patient table
location_data.to_csv('location.csv', index=False)              # Location table

# (Optional) Print preview
print("Cleaned Patient Data:")
print(patient_final.head())

print("\nLocation Data:")
print(location_data.head())


Cleaned Patient Data:
   pid     fname     lname         DOB     sex  LocationID
0    1     Abdul   Kuvalis  1995-04-23    Male           1
1    2  Adrienne    Graham  1953-06-19  Female           2
2    3        Ai   Gerlach  1996-07-10  Female           3
3    4      Aida     Marks  2016-03-20  Female           4
4    5    Althea  Schmeler  1974-07-28  Female           5

Location Data:
   LocationID              city          state
0           1  West Springfield  Massachusetts
1           2         Haverhill  Massachusetts
2           3         Worcester  Massachusetts
3           4         Haverhill  Massachusetts
4           5           Milford  Massachusetts


In [4]:
import pandas as pd

# Load form_encounter data
form_encounter = pd.read_csv('form_encounter.csv', encoding='latin1')

# Keep only the required columns (dropping onset_date and sensitivity)
form_encounter_cleaned = form_encounter[['id', 'pid', 'date', 'reason']].copy()

# Rename 'id' to 'encounter_id'
form_encounter_cleaned.rename(columns={'id': 'encounter_id'}, inplace=True)

# Save the cleaned form_encounter data
form_encounter_cleaned.to_csv('cleaned_form_encounter.csv', index=False)

# (Optional) Print preview
print("Cleaned Form Encounter Data:")
print(form_encounter_cleaned.head())


Cleaned Form Encounter Data:
   encounter_id  pid                 date  \
0             1    1  2002-05-07 05:44:10   
1             2    1  2011-06-05 05:44:10   
2             3    1  2013-06-16 05:44:10   
3             4    1  2014-06-22 05:44:10   
4             5    1  2014-12-16 06:54:14   

                                       reason  
0                       Encounter for problem  
1                Well child visit (procedure)  
2  General examination of patient (procedure)  
3  General examination of patient (procedure)  
4                       Encounter for problem  


In [5]:
import pandas as pd

# Step 1: Load prescriptions and encounters
prescriptions = pd.read_csv('prescriptions.csv', encoding='latin1')
form_encounter = pd.read_csv('cleaned_form_encounter.csv', encoding='latin1')

# Step 2: Prepare prescriptions
prescriptions_cleaned = prescriptions[['id', 'patient_id', 'date_added', 'dosage', 'drug', 'unit', 'route']].copy()

# Rename columns
prescriptions_cleaned.rename(columns={
    'id': 'prescriptionID',
    'patient_id': 'pid'
}, inplace=True)

# Step 3: Build a lookup dictionary {pid: encounter_id}
encounter_lookup = form_encounter.drop_duplicates(subset=['pid']).set_index('pid')['encounter_id'].to_dict()

# Step 4: Map encounter_id directly without expanding rows
prescriptions_cleaned['encounter_id'] = prescriptions_cleaned['pid'].map(encounter_lookup)

# Step 5: Save cleaned prescriptions
prescriptions_cleaned.to_csv('cleaned_prescriptions.csv', index=False)

# Optional preview
print("Fixed Cleaned Prescriptions:")
print(prescriptions_cleaned.head())


Fixed Cleaned Prescriptions:
   prescriptionID  pid           date_added  dosage  \
0               1    1  2015-05-29 23:44:10       1   
1               2    1  2016-03-28 07:39:01       1   
2               3    2  1959-02-17 15:26:14       1   
3               4    2  1972-09-05 17:26:14       1   
4               5    2  2020-09-25 15:26:14       1   

                                                drug  unit  route  \
0  Amoxicillin 250 MG / Clavulanate 125 MG Oral T...    10     16   
1                   Acetaminophen 325 MG Oral Tablet    11     16   
2    diphenhydrAMINE Hydrochloride 25 MG Oral Tablet    12     16   
3                         doxycycline hyclate 100 MG    13     16   
4                       lisinopril 10 MG Oral Tablet    14     16   

   encounter_id  
0             1  
1             1  
2            14  
3            14  
4            14  


In [6]:
# Step 1: Load vitals and cleaned prescriptions
form_vitals = pd.read_csv('form_vitals.csv', encoding='latin1')
prescriptions_final = pd.read_csv('cleaned_prescriptions.csv', encoding='latin1')

# Step 2: Prepare vitals
form_vitals_cleaned = form_vitals[['id', 'pid', 'date', 'weight', 'height', 'temperature', 'pulse', 'respiration']].copy()

# Drop rows with nulls
form_vitals_cleaned = form_vitals_cleaned.dropna(subset=['weight', 'height', 'temperature', 'pulse', 'respiration'])

# Rename 'id' to 'vital_id'
form_vitals_cleaned.rename(columns={'id': 'vital_id'}, inplace=True)

# Step 3: Build a lookup dictionary {pid: encounter_id}
encounter_lookup_vitals = prescriptions_final.drop_duplicates(subset=['pid']).set_index('pid')['encounter_id'].to_dict()

# Step 4: Map encounter_id directly
form_vitals_cleaned['encounter_id'] = form_vitals_cleaned['pid'].map(encounter_lookup_vitals)

# Step 5: Save cleaned vitals
form_vitals_cleaned.to_csv('cleaned_form_vitals.csv', index=False)

# Optional preview
print("Fixed Cleaned Form Vitals:")
print(form_vitals_cleaned.head())


Fixed Cleaned Form Vitals:
    vital_id  pid                 date  weight  height  temperature  pulse  \
4          5    5  2022-11-13 19:23:04  171.33   63.66       100.34   79.0   
6          7    7  2022-12-07 12:40:03  127.69   63.98        99.28   96.0   
25        26   28  2022-11-05 00:35:14  170.74   63.94        99.65   80.0   
49        50   56  2022-07-08 20:08:49  186.62   65.79        99.19   94.0   
55        56   62  2022-08-26 22:41:40  183.45   68.27        99.43   93.0   

    respiration  encounter_id  
4          13.0           181  
6          13.0           252  
25         14.0          1163  
49         15.0          2313  
55         14.0          2663  


In [7]:
import pandas as pd

# Step 1: Load procedure_order data
procedure_order = pd.read_csv('procedure_order.csv', encoding='latin1')

# Step 2: Keep only needed columns
procedure_order_cleaned = procedure_order[['procedure_order_id', 'encounter_id', 'date_ordered', 'order_status', 'activity', 'procedure_order_type']].copy()

# Step 3: Save the cleaned procedure_order
procedure_order_cleaned.to_csv('cleaned_procedure_order.csv', index=False)

# Optional: Print preview
print("Cleaned Procedure Order Data:")
print(procedure_order_cleaned.head())


Cleaned Procedure Order Data:
   procedure_order_id  encounter_id         date_ordered order_status  \
0                   1             4  2013-06-16 05:44:10    completed   
1                   2             4  2013-06-16 06:23:23    completed   
2                   3             4  2013-06-16 06:37:55    completed   
3                   4             4  2013-06-16 06:59:34    completed   
4                   5             4  2013-06-16 07:12:54    completed   

   activity procedure_order_type  
0         1                order  
1         1                order  
2         1                order  
3         1                order  
4         1                order  


In [8]:
import pandas as pd

# Load procedure_report data (Make sure procedure_report.csv is in the same folder)
procedure_report = pd.read_csv('procedure_report.csv', encoding='latin1')

# Keep only the needed columns
procedure_report_cleaned = procedure_report[['procedure_report_id', 'procedure_order_id', 'date_report', 'report_status', 'review_status']].copy()

# Save the cleaned procedure_report data
procedure_report_cleaned.to_csv('cleaned_procedure_report.csv', index=False)

# Optional: Preview
print("Cleaned Procedure Report Data:")
print(procedure_report_cleaned.head())


Cleaned Procedure Report Data:
   procedure_report_id  procedure_order_id          date_report report_status  \
0                    1                  27  2013-06-16 00:00:00         final   
1                    2                  28  2013-06-16 00:00:00         final   
2                    3                  29  2014-06-22 00:00:00         final   
3                    4                  30  2014-06-22 00:00:00         final   
4                    5                  31  2014-06-22 00:00:00         final   

  review_status  
0      reviewed  
1      reviewed  
2      reviewed  
3      reviewed  
4      reviewed  
